This notebook uses model trained in [this training notebook](https://www.kaggle.com/motloch/sorghum-pytorch-starter) to predict crop varietals for Sorghum -100 Cultivar Identification - FGVC 9. Based on Resnet34. Achieves 0.275 on the leaderboard.

Please upvote if you find this notebook useful. Thank you!

# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
import random
import os
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm
from torch.utils.data import Dataset, DataLoader
from torch import Tensor
from torchvision.transforms import ToTensor
import torch.nn as nn
from torchvision import transforms

# Load data

In [ ]:
train_df = pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv', index_col = 'image')

In [ ]:
images_present = os.listdir('../input/sorghum-id-fgvc-9/train_images')
train_df = train_df.loc[images_present]

In [ ]:
classes = train_df['cultivar'].unique()

In [ ]:
sample_sub = pd.read_csv('../input/sorghum-id-fgvc-9/sample_submission.csv', index_col = 'filename')

In [ ]:
X_test = sample_sub.index.values

# Set up Pytorch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
valid_bs = 8
num_workers = 2
model_arch = 'resnet34'
n_class = 100
x_size = 224
y_size = 224

# Data loader

In [ ]:
dir_name = '../input/sorghum-id-fgvc-9/test/'

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
class TestDataset(Dataset):
    def __init__(self):
        super().__init__()
    
    def __len__(self):
        return len(X_test)

    def __getitem__(self, idx):
        name = X_test[idx]
        x = Image.open(dir_name + name)
        x = test_transform(x)
        return x
    
test = TestDataset()
test_dl = DataLoader(test, batch_size = valid_bs, shuffle = False, num_workers = num_workers)

# Model

In [ ]:
class OurModel(nn.Module):
    def __init__(self, model_arch, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = OurModel(model_arch, pretrained = False)
model.load_state_dict(torch.load('../input/sorghum-pytorch-starter/epoch_4.pth'))
model.to('cuda')

# Predict class probabilities

In [ ]:
PREDS = []

model.eval()
with torch.no_grad():
    for i, x in enumerate(test_dl):
        x = x.to(device)
        logits = model(x)        
        PREDS += [logits.sigmoid()]

PREDS = torch.cat(PREDS).cpu().numpy()

# Predict classes and save

In [ ]:
argmaxes = np.argmax(PREDS, axis = 1)
predictions = [classes[a] for a in argmaxes]

In [ ]:
sample_sub['cultivar'] = predictions

In [ ]:
sample_sub.to_csv('submission.csv')